# 2. Backfill FRED Macroeconomic Data
Fetch historical data for 10-year Treasury yield and CPI

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
from utils.data_fetchers import fetch_fred_series
from utils.hopsworks_helpers import get_feature_store, create_feature_group
from dotenv import load_dotenv
import yaml

load_dotenv()

# Load config
with open('../config/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

## Fetch 10-Year Treasury Yield (DGS10)

In [ ]:
start_date = config['data']['start_date']
end_date = config['data']['end_date']

treasury_data = fetch_fred_series('DGS10', start_date, end_date)

print(f"Treasury yield data shape: {treasury_data.shape}")
treasury_data.head()

## Fetch CPI Data

In [ ]:
cpi_data = fetch_fred_series('CPIAUCSL', start_date, end_date)

print(f"CPI data shape: {cpi_data.shape}")
print(f"CPI is released monthly, so we have fewer observations")
cpi_data.head()

## Upload to Hopsworks Feature Store

In [ ]:
# Connect to Hopsworks
fs = get_feature_store()

# Create feature groups
treasury_fg = create_feature_group(
    fs,
    name='treasury_yield_raw',
    df=treasury_data,
    primary_key=['date'],
    description='Daily 10-year Treasury yield from FRED'
)

cpi_fg = create_feature_group(
    fs,
    name='cpi_raw',
    df=cpi_data,
    primary_key=['date'],
    description='Monthly CPI data from FRED (will be forward-filled)'
)

print("FRED data uploaded to Hopsworks!")